## LLM Experimentation with DB + LangChain

In [1]:
!pip install langchain 
!pip install langchain-experimental 
!pip install boto3 --upgrade 
!pip install s3fs --upgrade 
!pip install sqlalchemy-redshift psycopg2-binary  --upgrade
!pip install sqlalchemy --upgrade 


  Using cached langchain-0.0.339-py3-none-any.whl.metadata (16 kB)
  Using cached SQLAlchemy-2.0.23-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.2-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.0.66-py3-none-any.whl.metadata (10 kB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached yarl-1.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (28 kB)
  Using cached frozenlist-1.4.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
  Using cached aiosignal-1.3.

In [29]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 64.5 MB/s eta 0:00:0000:0100:01


In [4]:
import re
import boto3
import pandas as pd
import os
import sqlite3

from tqdm import tqdm
tqdm.pandas()

from IPython.display import display, HTML

from langchain.llms.bedrock import Bedrock

from langchain.prompts import PromptTemplate
from langchain.utilities import SQLDatabase

from langchain.chains import create_sql_query_chain

from prompts import *

In [15]:
QA_DATA_PATH = "s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined_20231121.csv"
COLUMNS_DESC_PATH = './columns_description.xml'

QUESTION = 'Count the total number of inpatients'

MODEL_ID = 'anthropic.claude-v2'
MODEL_CONFIG = {
    "max_tokens_to_sample": 2048,
    "temperature": 0,
    "top_p": 1,
    "top_k": 50,
}

LOAD_OUTPUTS = False

PROMPT_VERSION = 'V2'
PROMPT_SQLITE = PROMPT_V2_SQLITE
PROMPT_REDSHIFT = PROMPT_V2_REDSHIFT

OUTPUT_PATH = f'../outputs/llms/prompt_{PROMPT_VERSION}.csv'

#Redshift params
REDSHIFT_HOST = "redshift-synpuf-db-cluster-1.ckbiikj73zqo.us-east-1.redshift.amazonaws.com"
REDSHIFT_PORT = 5439
REDSHIFT_DATABASE = 'dev'
USERNAME = "awsuser"
PASSWORD = "Merck123"
DB_SCHEMA = 'rwdex_raw_synpuf'

REDSHIFT_ENDPOINT = f"redshift+psycopg2://{USERNAME}:{PASSWORD}@{REDSHIFT_HOST}:{REDSHIFT_PORT}/{REDSHIFT_DATABASE}"

In [6]:
bedrock_runtime = boto3.client('bedrock-runtime')

def get_bedrock_llm(model_id, model_config, verbose=False, bedrock_client=bedrock_runtime):
    llm = Bedrock(client=bedrock_runtime,
                  model_id=model_id, 
                  verbose=verbose,
                  model_kwargs=model_config)
    return llm


def extract_xml(tags, text):
    """Extract content enclosed in XML tags and remove newline symbols within the tag."""
    # First, extract content between the tags
    for tag in tags:
        xml_pattern = f"(?:<{tag}.*?>)(.*?)(?:<\\/{tag}>)"
        extracted_content = re.findall(xml_pattern, text, re.DOTALL)
        
        if extracted_content:
            # Then, remove newline symbols from the extracted content
            return extracted_content[0].replace('\n', ' ').strip()

    return None


def get_sql(row0, chain, db_engine='sqlite'):
    row = row0.copy()
    
    gt_exec_col = "query_corrected_with_list_executed"
    if db_engine =='sqlite':
        gt_exec_col = "query_corrected_sqlite_with_list_executed"
    
    question = row['question_with_list']
    
    sql = 'GT SQL Non-Executable'
    if row[gt_exec_col]:
        response = db_chain.invoke({"question": question})
        sql = extract_xml(tags=['sql'], text=response)
    
    row[f'sql_predicted_{db_engine}'] = sql
    return row


def print_gt_and_pred_sql(row, gt_column, pred_column):
    """Print both the ground-truth and predicted SQL queries."""

    gt_sql = row[gt_column]
    if 'sqlite' in gt_column:
        gt_sql = gt_sql.replace('rwdex_raw_synpuf.', '')

    print(f"QA ID: {row['qa_id']}")
    print('-'*30)

    display(HTML(f"<font color=blue>QUESTION: {row['question_with_list']}</font>"))
    print('-'*30)
    
    display(HTML(f"<font color=green>SQL(GT): {gt_sql}</font>"))
    print('-'*30)

    display(HTML(f"<font color=gray>SQL(PRED): {row[pred_column]}</font>"))
    print('*'*30)

In [4]:
#Load Column Descriptions
with open(COLUMNS_DESC_PATH,'r') as f:
    col_descriptions = f.readlines()[1:]
col_descriptions = ''.join(col_descriptions)
print(col_descriptions[:500])

<data>
  <row>
    <column_name>DESYNPUF_ID</column_name>
    <column_description>DESYNPUF: Beneficiary Code</column_description>
    <table_name>beneficiary_summary</table_name>
  </row>
  <row>
    <column_name>BENE_BIRTH_DT</column_name>
    <column_description>DESYNPUF: Date of birth</column_description>
    <table_name>beneficiary_summary</table_name>
  </row>
  <row>
    <column_name>BENE_DEATH_DT</column_name>
    <column_description>DESYNPUF: Date of death</column_description>
    <table


In [5]:
db = SQLDatabase.from_uri(REDSHIFT_ENDPOINT,  schema=f'{DB_SCHEMA}')#engine_args={'options': '-csearch_path={}'.format(DB_SCHEMA)})
llm = get_bedrock_llm(model_id=MODEL_ID, verbose=True, model_config=MODEL_CONFIG)

In [ ]:
#Get table names in DB
# db.get_usable_table_names()

In [ ]:
# db.table_info

In [6]:
prompt = PROMPT_REDSHIFT.replace('{db_schema}', DB_SCHEMA)
prompt = prompt.replace('{columns_description}', col_descriptions)
prompt = PromptTemplate.from_template(prompt)
db_chain = create_sql_query_chain(llm, db, prompt=prompt)

In [50]:
for i in dict(db_chain)['middle'][0]:
    print(i)
    print('\n\n\n ---- \n\n\n')

('input_variables', ['input', 'table_info', 'top_k'])



 ---- 



('input_types', {})



 ---- 



('output_parser', None)



 ---- 



('partial_variables', {})



 ---- 



('template', '\n\nHuman: You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which c

In [7]:
QUESTION = "select all inpatients in which their claim started in 2005"
response = db_chain.invoke({"question": QUESTION, 'db_schema': DB_SCHEMA})
print(response)

<sql>
SELECT DISTINCT "DESYNPUF_ID"
FROM rwdex_raw_synpuf."inpatient_claims"
WHERE "CLM_FROM_DT" BETWEEN '2005-01-01' AND '2005-12-31'
</sql>


In [17]:
def parse_string(string,tag):
    pattern = fr'<{tag}>(.*?)</{tag}>'
    # Find all matches in the string
    matches = re.findall(pattern, string,re.DOTALL)
    # return the first match
    try:
        return matches[0]
    except:
        return None
code = parse_string(response,"sql")
code.strip('\n')

'SELECT DISTINCT "DESYNPUF_ID"\nFROM rwdex_raw_synpuf."inpatient_claims"\nWHERE "CLM_FROM_DT" BETWEEN \'2005-01-01\' AND \'2005-12-31\''

In [54]:
try:
    db.run(code+'haha')
except Exception as e:
    ex = e


In [55]:
str(ex)

'(psycopg2.errors.SyntaxError) syntax error at or near "haha"\nLINE 5: haha\n        ^\n\n[SQL: \nSELECT DISTINCT "DESYNPUF_ID"\nFROM rwdex_raw_synpuf."inpatient_claims"\nWHERE "CLM_FROM_DT" BETWEEN \'2005-01-01\' AND \'2005-12-31\'\nhaha]\n(Background on this error at: https://sqlalche.me/e/14/f405)'

In [57]:
ex.orig

psycopg2.errors.SyntaxError('syntax error at or near "haha"\nLINE 5: haha\n        ^\n')

In [34]:
out = db.run(code)

In [36]:
out

''

In [31]:
type(ex)

sqlalchemy.exc.ProgrammingError

In [11]:
sql = extract_xml(tags=['sql'], text=response)
print(sql)

SELECT DISTINCT "DESYNPUF_ID" FROM rwdex_raw_synpuf."inpatient_claims" WHERE "CLM_FROM_DT" BETWEEN '2005-01-01' AND '2005-12-31'


### Run for all examples

In [12]:
df_qa = pd.read_csv(QA_DATA_PATH)
print(df_qa.shape)
df_qa.head()

(40, 13)


qa_id                                 question  \
0    t1          count the number of outpatients   
1    t2           count the number of inpatients   
2    t3       select all inpatients in year 2005   
3    t4      select all outpatients in year 2006   
4    t5  return total patients from 2015 to 2016   

                        question_with_list  \
0          count the number of outpatients   
1           count the number of inpatients   
2       select all inpatients in year 2005   
3      select all outpatients in year 2006   
4  return total patients from 2015 to 2016   

                                               query  \
0  select count(distinct desynpuf_id) from rwdex_...   
1  select count(distinct desynpuf_id) from rwdex_...   
2  select distinct desynpuf_id from rwdex_raw_syn...   
3  select distinct desynpuf_id from rwdex_raw_syn...   
4  select distinct desynpuf_id from rwdex_raw_syn...   

                                     query_corrected  \
0  select count(distinct desynpuf_id) from rwdex_...   
1  select count(distinct desynpuf_id) from rwdex_...   
2  select distinct desynpuf_id from rwdex_raw_syn...   
3  select distinct desynpuf_id from rwdex_raw_syn...   
4  select distinct desynpuf_id from rwdex_raw_syn...   

                           query_corrected_with_list  difficulty_level  \
0  select count(distinct desynpuf_id) from rwdex_...                 1   
1  select count(distinct desynpuf_id) from rwdex_...                 1   
2  select distinct desynpuf_id from rwdex_raw_syn...                 2   
3  select distinct desynpuf_id from rwdex_raw_syn...                 2   
4  select distinct desynpuf_id from rwdex_raw_syn...                 3   

  code_list_needed code_list_table  \
0              NaN             NaN   
1              NaN             NaN   
2              Yes             NaN   
3              Yes             NaN   
4              NaN             NaN   

                                       gpt4_response  \
0  SELECT COUNT(*) AS outpatient_count\nFROM rwde...   
1  SELECT COUNT(DISTINCT desynpuf_id) AS distinct...   
2  SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_sy...   
3  SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_sy...   
4  SELECT COUNT(DISTINCT desynpuf_id) AS total_pa...   

                                 gpt4_followup_input  \
0            I want the distinct numbers of patients   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  I want results from two tables: outpatient and...   

                               gpt4_revised_response  \
0  SELECT COUNT(DISTINCT desynpuf_id) AS distinct...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  SELECT 'Outpatient' AS patient_type, COUNT(DIS...   

   query_corrected_with_list_executed  
0                                True  
1                                True  
2                                True  
3                                True  
4                                True

In [13]:
# Count number of executable SQLs
df_qa.query_corrected_with_list_executed.value_counts()

True     30
False    10
Name: query_corrected_with_list_executed, dtype: int64

In [14]:
if LOAD_OUTPUTS:
    df_qa = pd.read_csv(OUTPUT_PATH)
else:
    df_qa = df_qa.progress_apply(get_sql, args=(db_chain, 'redshift'), axis=1)

100%|██████████| 40/40 [08:34<00:00, 12.87s/it]


In [15]:
#Display GT and predicted SQL
_ = df_qa.apply(print_gt_and_pred_sql, args=("query_corrected_with_list", "sql_predicted_redshift"), axis=1)

QA ID: t1
------------------------------


------------------------------


------------------------------


******************************
QA ID: t2
------------------------------


------------------------------


...

------------------------------


******************************
QA ID: t40
------------------------------


------------------------------


------------------------------


******************************


In [16]:
## Save Outputs
df_qa.to_csv(OUTPUT_PATH, index=False)

In [ ]:
"table_info": lambda x: db.get_table_info(
            table_names=x.get("table_names_to_use")
        ),

In [53]:
print(db.get_table_info())


CREATE TABLE rwdex_raw_synpuf.beneficiary_summary (
	desynpuf_id VARCHAR(256), 
	bene_birth_dt DATE, 
	bene_death_dt DATE, 
	bene_sex_ident_cd INTEGER, 
	bene_race_cd INTEGER, 
	bene_esrd_ind VARCHAR(256), 
	sp_state_code INTEGER, 
	bene_county_cd INTEGER, 
	bene_hi_cvrage_tot_mons INTEGER, 
	bene_smi_cvrage_tot_mons INTEGER, 
	bene_hmo_cvrage_tot_mons INTEGER, 
	plan_cvrg_mos_num INTEGER, 
	sp_alzhdmta INTEGER, 
	sp_chf INTEGER, 
	sp_chrnkidn INTEGER, 
	sp_cncr INTEGER, 
	sp_copd INTEGER, 
	sp_depressn INTEGER, 
	sp_diabetes INTEGER, 
	sp_ischmcht INTEGER, 
	sp_osteoprs INTEGER, 
	sp_ra_oa INTEGER, 
	sp_strketia INTEGER, 
	medreimb_ip REAL, 
	benres_ip REAL, 
	pppymt_ip REAL, 
	medreimb_op REAL, 
	benres_op REAL, 
	pppymt_op REAL, 
	medreimb_car REAL, 
	benres_car REAL, 
	pppymt_car REAL
)

/*
3 rows from beneficiary_summary table:
desynpuf_id	bene_birth_dt	bene_death_dt	bene_sex_ident_cd	bene_race_cd	bene_esrd_ind	sp_state_code	bene_county_cd	bene_hi_cvrage_tot_mons	bene_smi_cvrage_

In [3]:
COLUMNS_DESC_PATH = './columns_description.xml'

In [1]:
COLUMNS_DESC_PATH = './columns_description.xml'

In [2]:
with open(COLUMNS_DESC_PATH,'r') as f:
    col_descriptions = f.readlines()[1:]
col_descriptions = ''.join(col_descriptions)


In [16]:
llm = get_bedrock_llm(model_id=MODEL_ID, verbose=True, model_config=MODEL_CONFIG)

In [17]:
out = llm.predict(f"Convert this table to markdown format \n <table>{col_descriptions}</table>. Put it in <md> tags.")

In [18]:
print(out)

 <md>

| Column Name | Column Description | Table Name |
|-|-|-|  
| DESYNPUF_ID | DESYNPUF: Beneficiary Code | beneficiary_summary |
| BENE_BIRTH_DT | DESYNPUF: Date of birth | beneficiary_summary |
| BENE_DEATH_DT | DESYNPUF: Date of death | beneficiary_summary |
| BENE_SEX_IDENT_CD | DESYNPUF: Sex | beneficiary_summary |
| BENE_RACE_CD | DESYNPUF: Beneficiary Race Code | beneficiary_summary |
| BENE_ESRD_IND | DESYNPUF: End stage renal disease Indicator | beneficiary_summary |
| SP_STATE_CODE | DESYNPUF: State Code | beneficiary_summary |
| BENE_COUNTY_CD | DESYNPUF: County Code | beneficiary_summary |
| BENE_HI_CVRAGE_TOT_MONS | DESYNPUF: Total number of months of part A coverage for the beneficiary. | beneficiary_summary |
| BENE_SMI_CVRAGE_TOT_MONS | DESYNPUF: Total number of months of part B coverage for the beneficiary. | beneficiary_summary |
| BENE_HMO_CVRAGE_TOT_MONS | DESYNPUF: Total number of months of HMO coverage for the beneficiary. | beneficiary_summary |
| PLAN_CVRG_M

In [19]:
import pandas as pd

In [23]:
df = pd.read_xml("columns_description.xml")

In [25]:
df.to_markdown()

'|    | column_name                       | column_description                                                            | table_name               |\n|---:|:----------------------------------|:------------------------------------------------------------------------------|:-------------------------|\n|  0 | DESYNPUF_ID                       | DESYNPUF: Beneficiary Code                                                    | beneficiary_summary      |\n|  1 | BENE_BIRTH_DT                     | DESYNPUF: Date of birth                                                       | beneficiary_summary      |\n|  2 | BENE_DEATH_DT                     | DESYNPUF: Date of death                                                       | beneficiary_summary      |\n|  3 | BENE_SEX_IDENT_CD                 | DESYNPUF: Sex                                                                 | beneficiary_summary      |\n|  4 | BENE_RACE_CD                      | DESYNPUF: Beneficiary Race Code                   

In [22]:
! pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.5 MB/s eta 0:00:00:00:0100:01


In [26]:
for tab in reversed(range(0, 10)):
    print(tab)


9
8
7
6
5
4
3
2
1
0


In [28]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.2 MB/s eta 0:00:00


<table>
    <bene_summary>
<columns>
    